In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import numpy as np
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import os
import seaborn as sns

In [20]:
from utils.pipeline import run_matches_discovery, discovery_pipeline
from utils.pipeline_wrappers import gen_expname, run_until_coverage_th, try_run_exp, cv_experiment, monitor_callback, grid_exp
from utils.feature_utils import op_100, get_feats, get_seq_names, get_dh_feats_wrap, load_feats
from utils.helper_fncs import load_json
from utils.skopt_tune import TunerSkopt
from copy import deepcopy

In [50]:
nfolder_persigner = [1475,
                    49,
                    470,
                    836,
                    1296,
                    30,
                    646,
                    704,
                    165]

report_cols = ['ned','length_avg', 'n_clus', 'n_node',
              'grouping_P', 'grouping_R', 'grouping_F', 
              'nfile', 'coverageNS']

signers_per_set = {'A': [4,8,9], 'B':[2,5,7], 'C':[1,3,6]}

In [51]:
nfolder_perset = dict()
for devset in ['A','B','C']:
    nfolder_perset[devset] = 0
    for i in signers_per_set[devset]:
        key = 'Signer0{}'.format(i) 
        nfolder_perset[key] = nfolder_persigner[i-1]
        nfolder_perset[devset] += nfolder_persigner[i-1]
nfolder_perset

{'A': 1705,
 'Signer04': 836,
 'Signer08': 704,
 'Signer09': 165,
 'B': 1991,
 'Signer02': 49,
 'Signer05': 1296,
 'Signer07': 646,
 'C': 1975,
 'Signer01': 1475,
 'Signer03': 470,
 'Signer06': 30}

In [293]:
df = pd.read_csv('../results/clustering_zr_op100_modularity.csv')
#df = pd.read_csv('../results/cv/knn_op100_l6_lmax_dim_fix_r_s.csv')
#df = pd.read_csv('../results/clustering_zr_c3_custom.csv')
#df = pd.read_csv('../results/clustering_knn_ccomp.csv')
#df = pd.read_csv('../results/cv/zr_cat_op100.csv')


In [294]:
len(df)

9

In [295]:
df['nfile'] = df.set.apply(lambda x: nfolder_perset[x])


In [296]:
df.columns

Index(['Unnamed: 0', 'devset', 'set', 'ned', 'coverage', 'coverageNS',
       'coverageNS_f', 'grouping_F', 'grouping_P', 'grouping_R', 'token_F',
       'token_P', 'token_R', 'type_F', 'type_P', 'type_R', 'boundary_F',
       'boundary_P', 'boundary_R', 'n_clus', 'n_node', 'exp_path',
       'length_avg', 'nfile'],
      dtype='object')

In [297]:
df.head()

,Unnamed: 0,devset,set,ned,coverage,coverageNS,coverageNS_f,grouping_F,grouping_P,grouping_R,...,type_P,type_R,boundary_F,boundary_P,boundary_R,n_clus,n_node,exp_path,length_avg,nfile
0,0,A,A,87.08,10.27,10.56,14.08,3.78,1.95,66.67,...,0.33,0.14,6.40,100.0,3.31,23,308,/home/korhan/Desktop/tez/zr_exps/zr_cat_A_op10...,85.122257,1705
1,1,A,B,89.36,9.57,9.74,11.76,3.17,1.64,50.00,...,2.50,1.08,5.90,100.0,3.04,32,366,/home/korhan/Desktop/tez/zr_exps/zr_cat_B_op10...,72.997290,1991
2,2,A,C,95.58,10.61,10.88,13.42,2.34,1.21,38.46,...,1.24,0.63,6.71,100.0,3.48,25,413,/home/korhan/Desktop/tez/zr_exps/zr_cat_C_op10...,68.713953,1975
3,3,B,A,92.17,9.01,9.30,13.03,4.77,2.44,100.00,...,0.00,0.00,8.37,100.0,4.37,42,409,/home/korhan/Desktop/tez/zr_exps/zr_cat_A_op10...,59.336516,1705
4,4,B,B,93.04,9.81,10.16,13.29,4.15,2.16,57.14,...,1.29,0.84,8.81,100.0,4.61,66,556,/home/korhan/Desktop/tez/zr_exps/zr_cat_B_op10...,53.471429,1991


In [298]:
devs_ms = []
devs_si = []
tests_ms = []
tests_si = []

In [299]:
for devset in ['A','B','C']:
    dev_signers = ['Signer0{}'.format(i) for i in signers_per_set[devset]]

    fold_df = df.loc[df.devset.isin([devset])]
    dev_df = fold_df.loc[fold_df.set.isin([devset]+dev_signers)][report_cols+['set']]
    test_df = fold_df.loc[~fold_df.set.isin([devset]+dev_signers)][report_cols+['set']]

    fold = dev_df
    fold_sd = fold.loc[[len(x)>1 for x in fold.set]]
    fold_ms = fold.loc[[len(x)==1 for x in fold.set]]
    tmp = pd.DataFrame(((fold_sd[report_cols].values * fold_sd.nfile.values[:,None]).sum(0) / fold_sd.nfile.sum())[None,:],
             columns=report_cols)
    
    devs_ms.append(fold_ms)
    devs_si.append(tmp)
    
    fold = test_df
    fold_sd = fold.loc[[len(x)>1 for x in fold.set]]
    fold_ms = fold.loc[[len(x)==1 for x in fold.set]]
    tmp = pd.DataFrame(((fold_sd[report_cols].values * fold_sd.nfile.values[:,None]).sum(0) / fold_sd.nfile.sum())[None,:],
             columns=report_cols)
    
    tests_ms.append(fold_ms)
    tests_si.append(tmp)


/home/korhan/anaconda3/envs/tez/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
/home/korhan/anaconda3/envs/tez/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


In [300]:
def averege(df_list):
    fold_sd = pd.concat(df_list)
    tmp = pd.DataFrame(((fold_sd[report_cols].values * fold_sd.nfile.values[:,None]).sum(0) / fold_sd.nfile.sum())[None,:],
                 columns=report_cols)
    return tmp

In [301]:
devs_ms

[     ned  length_avg  n_clus  n_node  grouping_P  grouping_R  grouping_F  \
 0  87.08   85.122257      23     308        1.95       66.67        3.78   
 
    nfile  coverageNS set  
 0   1705       10.56   A  ,
      ned  length_avg  n_clus  n_node  grouping_P  grouping_R  grouping_F  \
 4  93.04   53.471429      66     556        2.16       57.14        4.15   
 
    nfile  coverageNS set  
 4   1991       10.16   B  ,
      ned  length_avg  n_clus  n_node  grouping_P  grouping_R  grouping_F  \
 8  94.81   51.850598      43     492        3.05       71.43        5.84   
 
    nfile  coverageNS set  
 8   1975        9.17   C  ]

In [302]:
keys = ['Dev MS' , 'Dev SD' ,'Test MS' ,'Test SD']

final = []

for k,df_list in enumerate([devs_ms , devs_si ,tests_ms ,tests_si ]):
    avg_df = averege(df_list)
    avg_df['exp'] = keys[k]
    
    final.append(avg_df)
    
    display(avg_df)

,ned,length_avg,n_clus,n_node,grouping_P,grouping_R,grouping_F,nfile,coverageNS,exp
0,91.864537,62.422852,45.061894,459.149356,2.406817,64.981897,4.627323,1899.441192,9.935481,Dev MS


,ned,length_avg,n_clus,n_node,grouping_P,grouping_R,grouping_F,nfile,coverageNS,exp
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dev SD


,ned,length_avg,n_clus,n_node,grouping_P,grouping_R,grouping_F,nfile,coverageNS,exp
0,93.008799,60.665281,39.494534,458.27526,2.168222,55.164131,4.143783,1899.441192,10.036944,Test MS


,ned,length_avg,n_clus,n_node,grouping_P,grouping_R,grouping_F,nfile,coverageNS,exp
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test SD


In [304]:
findf = pd.concat(final)[['exp'] + report_cols]
#findf[['grouping_P','grouping_R','grouping_F','ned']] *= 100
findf

,exp,ned,length_avg,n_clus,n_node,grouping_P,grouping_R,grouping_F,nfile,coverageNS
0,Dev MS,91.864537,62.422852,45.061894,459.149356,2.406817,64.981897,4.627323,1899.441192,9.935481
0,Dev SD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Test MS,93.008799,60.665281,39.494534,458.275260,2.168222,55.164131,4.143783,1899.441192,10.036944
0,Test SD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
